In [ ]:
import numpy as np

In [133]:
theta = np.array([1,2,3,4,5])
phi = np.array([2,2,2,2,2])

In [ ]:
np.dot(theta.T, phi)

In [47]:
np.random.uniform(low=0.1, high=0.5, size=2)

array([ 0.43941165,  0.48912863])

In [ ]:
len(theta)

In [1]:
import cv2
import numpy as np

from enduro.agent import Agent
from enduro.action import Action

In [37]:
np.random.choice(4)

2

In [46]:
y = [phi_position(x), phi_action('BRAKE')]
y

[3, -3]

In [170]:
np.where(grid_opponents==1)[1]

array([5, 6, 5])

In [212]:
_ = np.where(grid_opponents==1)[1]-np.where(x==2)[0][0]
if any(x<=0 and x>-3 for x in _):
    print _

[-2  2]


In [292]:
def phi_position(grid, act):
    grid = grid.tolist()
    if grid.index(2)>5:
        if act == 0:
            return 1
        elif act == 2:
            return 5
        else:
            return 1
    elif grid.index(2)<4:
        if act == 0:
            return 1
        elif act == 1:
            return 5
        else:
            return 1
    else:
        if act == 0:
            return 10
        elif act == 3:
            return 1
        else:
            return 2

def phi_action(act):
    return 0
    if act == 1 or act == 2:
        return 2
    elif act == 0:
        return 5
    else:
        return 1

def phi_opponent(grid_agent, grid_opponent, act):
    opponents = np.where(grid_opponent==1)[1]
    agent = np.where(grid_agent==2)[0][0]
    
    if any(x>=0 and x<3 for x in opponents-agent):
        #print "WARNING"
        #print opponents
        #print agent
        if act == 2:
            return 20
        else:
            return 0
    elif any(x<=0 and x>-3 for x in opponents-agent):
        #print "WARNING 2"
        #print opponents
        #print agent
        if act == 1:
            return 20
        else:
            return 0      
    else:
        if act == 3:
            return 0
        else:
            return 5

In [293]:
class FunctionApproximationAgent(Agent):
    def __init__(self):
        super(FunctionApproximationAgent, self).__init__()
        # The horizon defines how far the agent can see
        self.horizon_row = 5

        self.grid_cols = 10
        # The state is defined as a tuple of the agent's x position and the
        # x position of the closest opponent which is lower than the horizon,
        # if any is present. There are four actions and so the Q(s, a) table
        # has size of 10 * (10 + 1) * 4 = 440.
        self.Q = np.ones((self.grid_cols, self.grid_cols + 1, 4))

        # Add initial bias toward moving forward. This is not necessary,
        # however it speeds up learning significantly, since the game does
        # not provide negative reward if no cars have been passed by.
        self.Q[:, :, 0] += 1.

        # Helper dictionaries that allow us to move from actions to
        # Q table indices and vice versa
        self.idx2act = {i: a for i, a in enumerate(self.getActionsSet())}
        self.act2idx = {a: i for i, a in enumerate(self.getActionsSet())}

        # Learning rate
        self.alpha = 0.00001
        # Discounting factor
        self.gamma = 0.9
        # Exploration rate
        self.epsilon = 0.01

        # Log the obtained reward during learning
        self.last_episode = 1
        self.episode_log = np.zeros(6510) - 1.
        self.log = []
        
        self.theta = np.random.uniform(low=0.1, high=0.5, size=2)
        self.theta = np.array([0.1,0.1,0.1])
        self.last_action = 0
        self.total_reward = 0

    def initialise(self, road, cars, speed, grid):
        """ Called at the beginning of an episode. Use it to construct
        the initial state.
        """
        print 'Total reward = '+str(self.total_reward)+"; Theta = "+str(self.theta)
        self.total_reward = 0
        self.next_state2 = grid[:5]

    def act(self):
        """ Implements the decision making process for selecting
        an action. Remember to store the obtained reward.
        """

        self.state2 = self.next_state2

        # If exploring
        if np.random.uniform(0., 1.) < self.epsilon:
            # Select a random action using softmax
            idx = np.random.choice(4)#, p=probs)
            self.action = self.idx2act[idx]
            self.last_action = idx
        else:
            # Select the greedy action
            idx = self.argmax_Qsa(self.state2)
            #print idx
            self.action = self.idx2act[idx]
            self.last_action = idx

        self.reward = self.move(self.action)
        self.total_reward += self.reward

    def sense(self, road, cars, speed, grid):
        """ Constructs the next state from sensory signals.

        Args:
            road  -- 2-dimensional array containing [x, y] points
                     in pixel coordinates of the road grid
            cars  -- dictionary which contains the location and the size
                     of the agent and the opponents in pixel coordinates
            speed -- the relative speed of the agent with respect the others
            gird  -- 2-dimensional numpy array containing the latest grid
                     representation of the environment

        For more information on the arguments have a look at the README.md
        """
        self.next_state2 = grid[:5]

        # Visualise the environment grid
        #cv2.imshow("Enduro", self._image)

    def learn(self):
        """ Performs the learning procedure. It is called after act() and
        sense() so you have access to the latest tuple (s, s', a, r).
        """
        #print "last action = "+str(self.last_action)
        #print "state2 = "+str(self.state2)
        _max_Qsa = self.max_Qsa(self.next_state2)
        self.phi = [phi_position(self.state2[0], self.last_action), phi_action(self.last_action),
                   phi_opponent(self.state2[0], self.state2[1:], self.last_action)]
        
        Q_sa = np.dot(self.theta.T, self.phi)
        #print str(Q_sa)+" = "+str(self.theta)+' * '+str(self.phi)
        
        
        '''print "grad = " + str(np.dot(self.alpha*(self.reward + self.gamma * _max_Qsa - Q_sa),
                                    self.phi)) + " = " + str(
            self.alpha*(self.reward + self.gamma * _max_Qsa - Q_sa)) + " * " + str(
        self.phi)'''
        
        self.theta = self.theta + np.dot(self.alpha*(self.reward + self.gamma * _max_Qsa - Q_sa), self.phi)
        
        #print "Updated theta = "+str(self.theta)
        
        #Q_sa = self.Q[self.state[0], self.state[1], self.act2idx[self.action]]

        # Calculate the updated state action value
        #Q_sa_new = Q_sa + self.alpha * (self.reward + self.gamma * self.maxQsa(self.next_state) - Q_sa)

        # Write the updated value
        #self.Q[self.state[0], self.state[1], self.act2idx[self.action]] = Q_sa_new
        pass

    def callback(self, learn, episode, iteration):
        """ Called at the end of each timestep for reporting/debugging purposes.
        """
        #if not iteration%1000:
        #    print "{0}/{1}: {2}".format(episode, iteration, self.total_reward)

        #if not episode % 100:
        cv2.imshow("Enduro", self._image)
        #cv2.waitKey(1)

    def QsaAll(self, grid):
        self.allQsa = []
        for each in [0,1,2,3]:
            self.phi = [phi_position(grid[0], each), phi_action(each), 
                        phi_opponent(grid[0], grid[1:], each)]
            _ = np.dot(self.theta.T, self.phi)
            self.allQsa.append(_)
        #print self.allQsa
        #print 'MAX = ' + str(np.max(self.allQsa))
        #print 'ARD MAX = ' + str(np.argmax(self.allQsa))
        return self.allQsa
    
    def max_Qsa(self, grid):
        return np.max(self.QsaAll(grid))
    def argmax_Qsa(self, grid):
        return np.argmax(self.QsaAll(grid))
    
    def maxQsa(self, state):
        return np.max(self.Q[state[0], state[1], :])

    def argmaxQsa(self, state):
        return np.argmax(self.Q[state[0], state[1], :])

if __name__ == "__main__":
    a = FunctionApproximationAgent()
    a.run(True, episodes=2000, draw=True)
    print 'Total reward: ' + str(a.total_reward)

Total reward = 0; Theta = [ 0.1  0.1  0.1]
Total reward = 122; Theta = [ 0.09813011  0.1         0.08567354]
Total reward = 112; Theta = [ 0.09436119  0.1         0.07382267]
Total reward = 99; Theta = [ 0.09043657  0.1         0.06597611]
Total reward = 88; Theta = [ 0.08677916  0.1         0.05937009]
Total reward = 87; Theta = [ 0.08352485  0.1         0.05256415]
Total reward = 80; Theta = [ 0.08009939  0.1         0.04732148]
Total reward = 83; Theta = [ 0.07700094  0.1         0.04282258]
Total reward = 93; Theta = [ 0.07452787  0.1         0.04085865]
Total reward = 78; Theta = [ 0.07170716  0.1         0.03765769]
Total reward = 132; Theta = [ 0.07258508  0.1         0.03816399]
Total reward = 88; Theta = [ 0.07010158  0.1         0.0353252 ]
Total reward = 133; Theta = [ 0.07073339  0.1         0.03621661]
Total reward = 94; Theta = [ 0.06960671  0.1         0.03443281]
Total reward = 116; Theta = [ 0.069226    0.1         0.03519221]
Total reward = 85; Theta = [ 0.0673717   0

KeyboardInterrupt: 

1/2000: 5
1/3000: 14


KeyboardInterrupt: 